<a href="https://colab.research.google.com/github/mihirgupte/sentence-boundary-disambiguation/blob/main/sentence_boundary_disambiguation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import re

## Initialising the dataset

In [2]:
#print(os.listdir("/content/"))
text = []

for files in os.listdir():
  if files==".config" or files==".ipynb_checkpoints" or files == 'sample_data':
    continue
  with open(files, encoding = 'utf-8') as f:
    sent = f.readlines()
    text.extend(sent)

In [3]:
#text = [text[i]+" "+text[i+1] for i in range(0,len(text),2)]
text

['\n',
 '\n',
 '\tIn/in a/at few/ap school/nn districts/nns one/pn finds/vbz a/at link/nn between/in school/nn and/cc job/nn ./.\n',
 'In/in those/dts vocational/jj programs/nns organized/vbn with/in Smith-Hughes/np money/nn ,/, there/ex may/md be/be a/at close/jj tie/nn between/in the/at labor/nn union/nn and/cc a/at local/jj employer/nn on/in the/at one/cd hand/nn and/cc the/at vocational/jj teacher/nn on/in the/at other/ap ./.\n',
 'In/in these/dts cases/nns a/at graduate/nn may/md enter/vb directly/rb into/in an/at apprentice/nn program/nn ,/, saving/vbg a/at year/nn because/rb of/in his/pp$ vocational/jj courses/nns in/in grades/nns 11/cd and/cc 12/cd ./.\n',
 'The/at apprentice/nn program/nn will/md involve/vb further/jjr education/nn on/in a/at part-time/jj basis/nn ,/, usually/rb at/in night/nn ,/, perhaps/rb using/vbg some/dti of/in the/at same/ap equipment/nn of/in the/at high/jj school/nn ./.\n',
 'These/dts opportunities/nns are/ber to/to be/be found/vbn in/in certain/jj ci

In [4]:
text = [i for i in text if i!='\n']
text = [i.replace("\n","").replace("\t","").strip() for i in text]
new_text = []

for i in text:
  words = i.split(" ")
  for word in range(len(words)):
    if "/." in words[word]:
      words[word]+="/Y"
    else:
      words[word]+="/N"
  new_text.append(" ".join(words))
new_text = [new_text[i]+" "+new_text[i+1] for i in range(0,len(new_text),2)]
new_text

['In/in/N a/at/N few/ap/N school/nn/N districts/nns/N one/pn/N finds/vbz/N a/at/N link/nn/N between/in/N school/nn/N and/cc/N job/nn/N ././Y In/in/N those/dts/N vocational/jj/N programs/nns/N organized/vbn/N with/in/N Smith-Hughes/np/N money/nn/N ,/,/N there/ex/N may/md/N be/be/N a/at/N close/jj/N tie/nn/N between/in/N the/at/N labor/nn/N union/nn/N and/cc/N a/at/N local/jj/N employer/nn/N on/in/N the/at/N one/cd/N hand/nn/N and/cc/N the/at/N vocational/jj/N teacher/nn/N on/in/N the/at/N other/ap/N ././Y',
 'In/in/N these/dts/N cases/nns/N a/at/N graduate/nn/N may/md/N enter/vb/N directly/rb/N into/in/N an/at/N apprentice/nn/N program/nn/N ,/,/N saving/vbg/N a/at/N year/nn/N because/rb/N of/in/N his/pp$/N vocational/jj/N courses/nns/N in/in/N grades/nns/N 11/cd/N and/cc/N 12/cd/N ././Y The/at/N apprentice/nn/N program/nn/N will/md/N involve/vb/N further/jjr/N education/nn/N on/in/N a/at/N part-time/jj/N basis/nn/N ,/,/N usually/rb/N at/in/N night/nn/N ,/,/N perhaps/rb/N using/vbg/N som

In [5]:
len(new_text)

13399

## Features

Check if words contains ".","?","!",";"

In [6]:
def feature1(words,index):
  exp = r"."
  if re.findall(r"[.!;?]$", words[index].split("/")[0]):
    return 1
  return 0

Check if word is at end of sentence

In [7]:
def feature2(words,index):
  #print(len(words))
  if len(words)<=(index+1):
    return 1
  return 0

Check for initials or abbreviations, eg. "A. B." or "e.g."

In [8]:
def feature3(words, index):
  pattern = re.compile(r"^[a-zA-Z][.]")
  pattern2 = re.compile(r"[a-zA-Z][.]([a-zA-Z][.])+")
  s = re.match(pattern,words[index])
  if index!=len(words)-1 and s != None:
    next_word = words[index+1]
    if re.match(pattern,next_word):
      return 1
  if index!=0 and s != None:
    prev_word = words[index-1]
    if re.match(pattern,prev_word):
      return 1
  if re.match(pattern2,words[index])!=None:
    return 1
  return 0

Check if word next to ".",";","?","!" is in uppercase(the first letter) and this word is in lowercase(the first letter).

In [9]:
def feature4(words,index):
  pattern = re.compile(r"[.!;?]$")
  if pattern.search(words[index].split("/")[0]) and index<len(words)-1:
    next_word = words[index+1].strip()[0]
    prev_word = words[index-1].strip()[0]
    if next_word.isupper() and prev_word.islower():
      return 1
  return 0

## Inserting dataset records and respective features in a DataFrame

In [10]:
sample = []
print("Outliers:")
for i in new_text:
  #sample.append([])
  l = []
  words = i.split(" ")
  for j in range(len(words)):
    try:
      word,_,target = words[j].split("/")
    except:
      print(words[j])
      continue
    l.append(word)
    l.append(feature1(words,j))
    l.append(feature2(words,j))
    l.append(feature3(words,j))
    l.append(feature4(words,j))
    l.append(target)
    sample.append(l)
    l = []
#sample

Outliers:
2/c/np-tl/N
1/c/np-tl/N
1/c/np-tl/N
17-1/2-inch/jj/N
87-1/2/cd/N
and/or/cc/N
1596/7/cd/N
1597/8/cd/N
1598/9/cd/N
1600/1/cd/N
9-1/2/cd-tl/N
1/4''/nn/N
1/4''/nn/N
1/2''/nn/N
1/4''/nn/N
3/16''/nn/N
1/2''/nn/N
1/4''/nn/N
3/16''/nn/N
1/2''/nn/N
3-1/2''/nns/N
1628/29/cd/N
4-7/8/cd/N
4-1/2%/nn/N
2-3/4/cd/N
2-1/2/cd/N
1-1/2/cd/N
and/or/cc/N
2-1/2/cd/N
1-1/2/cd/N
1/4/cd/N
1/4/cd/N
1/2/cd/N
1/4/cd/N
1/2/cd/N
1/8-inch/nn/N
1/2-inch/nn/N
5/8-inch/nn/N
3/8-inch/nn/N
1/8-inch/nn/N
$.30/mbf/nns/N
bd/ft/nn/N
$.07/cwt/nns/N
$4/mbf/nns/N
$.054/mbf/nns/N
$2.30/mbf/nns/N
1/50th/od/N
1/20th/od/N
1/8''/nn/N
1/4''/nn/N
1/2''/nn/N
5-3/4''/nns/N
1/2''/nn/N
1/2''/nn/N
1/2''/nn/N
5-3/4''/nns/N
1/8''/nn/N
3-3/4''/nns/N
1/2''/nn/N
3-3/4''/nns/N
1-1/4''/nn/N
1/4''/nn/N
5/64''/nn/N
21/64''/nns/N
1/4''/nn/N
2-3/4''/nns/N
7/16''/nn/N
1/8''/nn/N
1/8''/nn/N
9/32''/nn/N
5/16''/nn/N
1/8''/nn/N
1/8''/nn/N
1/8''/nn/N
3/64''/nn/N
and/or/cc/N
and/or/cc/N
3-1/2/cd/N
8-1/2-foot/jj/N
13/16-inch/jj/N
3/8-inch/nn/N
13/16

In [11]:
df = pd.DataFrame(sample,columns=['Word','Feature1','Feature2','Feature3','Feature4','Target'])
df

,Word,Feature1,Feature2,Feature3,Feature4,Target
0,In,0,0,0,0,N
1,a,0,0,0,0,N
2,few,0,0,0,0,N
3,school,0,0,0,0,N
4,districts,0,0,0,0,N
...,...,...,...,...,...,...
594036,of,0,0,0,0,N
594037,bread,0,0,0,0,N
594038,spread,0,0,0,0,N
594039,out,0,0,0,0,N


## More preprocessing

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Target'] = le.fit_transform(df.loc[:,'Target'])
df

,Word,Feature1,Feature2,Feature3,Feature4,Target
0,In,0,0,0,0,0
1,a,0,0,0,0,0
2,few,0,0,0,0,0
3,school,0,0,0,0,0
4,districts,0,0,0,0,0
...,...,...,...,...,...,...
594036,of,0,0,0,0,0
594037,bread,0,0,0,0,0
594038,spread,0,0,0,0,0
594039,out,0,0,0,0,0


In [13]:
np.sum(df['Feature4']==1)

9541

In [14]:
R1 = (np.sum(df['Target']==1))
R0 = (np.sum(df['Target']==0))
R1/R0

0.049544257145330645

In [15]:
X = df.iloc[:,1:-1].values
y = df.iloc[:,-1].values
print(X,y)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]] [0 0 0 ... 0 0 1]


In [16]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)
for train_index, test_index in skf.split(X, y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

TRAIN: [295469 295497 295526 ... 594038 594039 594040] TEST: [     0      1      2 ... 297106 297107 297108]
TRAIN: [     0      1      2 ... 297106 297107 297108] TEST: [295469 295497 295526 ... 594038 594039 594040]


## Training Models on dataset

In [17]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10,max_depth=5)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
preds = rf.predict(X_train)
preds

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
from sklearn.metrics import confusion_matrix,accuracy_score

print(confusion_matrix(preds,y_train))
print(accuracy_score(preds,y_train))

[[281582     20]
 [  1418  14001]]
0.9951585914800637


In [20]:
preds_test = rf.predict(X_test)
print(confusion_matrix(preds_test,y_test))
print(accuracy_score(preds_test,y_test))

[[281815      8]
 [  1184  14013]]
0.9959868022355397


## Making a SBD Function for real sentence prediction

In [23]:
def sbd(sentence):
  words = sentence.split(" ")
  sent = []
  for i in range(len(words)):
    f1 = feature1(words,i)
    f2 = feature2(words,i)
    f3 = feature3(words,i)
    f4 = feature4(words,i)
    feature_arr = np.array([[f1,f2,f3,f4]])
    pred = rf.predict(feature_arr)
    label = le.inverse_transform(pred)
    sent.append((words[i]+"/"+label[0]))
  return " ".join(sent)

In [24]:
s = 'In common parlance, randomness is the apparent or actual lack of pattern or predictability in events . A random sequence of events, symbols or steps often has no order and does not follow an intelligible pattern or combination . Individual random events are, by definition, unpredictable, but if the probability distribution is known, the frequency of different outcomes over repeated events (or "trials") is predictable. For example, when throwing two dice, the outcome of any particular roll is unpredictable, but a sum of 7 will tend to occur twice as often as 4. In this view, randomness is not haphazardness ; it is a measure of uncertainty of an outcome. Randomness applies to concepts of chance, probability, and information entropy .'
sbd(s)

'In/N common/N parlance,/N randomness/N is/N the/N apparent/N or/N actual/N lack/N of/N pattern/N or/N predictability/N in/N events/N ./Y A/N random/N sequence/N of/N events,/N symbols/N or/N steps/N often/N has/N no/N order/N and/N does/N not/N follow/N an/N intelligible/N pattern/N or/N combination/N ./Y Individual/N random/N events/N are,/N by/N definition,/N unpredictable,/N but/N if/N the/N probability/N distribution/N is/N known,/N the/N frequency/N of/N different/N outcomes/N over/N repeated/N events/N (or/N "trials")/N is/N predictable./Y For/N example,/N when/N throwing/N two/N dice,/N the/N outcome/N of/N any/N particular/N roll/N is/N unpredictable,/N but/N a/N sum/N of/N 7/N will/N tend/N to/N occur/N twice/N as/N often/N as/N 4./Y In/N this/N view,/N randomness/N is/N not/N haphazardness/N ;/Y it/N is/N a/N measure/N of/N uncertainty/N of/N an/N outcome./Y Randomness/N applies/N to/N concepts/N of/N chance,/N probability,/N and/N information/N entropy/N ./Y'